In [1]:
import sys
sys.path.append('../../../src')
import os
os.environ['GLOG_v'] = '4'
os.environ['MS_JIT_MODULES'] = 'sponge'

In [2]:
from mindspore import context
from mindspore.nn import Adam
from sponge import Sponge, Molecule, ForceField, UpdaterMD
from sponge import WithEnergyCell, set_global_units
from sponge.potential import SphericalRestrict
from sponge.function import VelocityGenerator
from sponge.callback import WriteH5MD, RunInfo

In [3]:
set_global_units('nm', 'kj/mol')
# context.set_context(mode=context.GRAPH_MODE, device_target='Ascend', device_id=0)
context.set_context(mode=context.GRAPH_MODE, device_id=0)

## 系统的扩展

如果使用reduplicate，系统会在内部进行扩展，但是如果使用copy，则会产生一个新的分子系统。也因此，连续扩展6次之后的系统，具有$2^6=64$个水分子，也就是192个原子。

In [4]:
system = Molecule(template='water.tip3p.yaml')
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
system.reduplicate([0, 0, 0.3])
system.reduplicate([-0.6, 0, 0])
system.reduplicate([0, -0.6, 0])
system.reduplicate([0, 0, -0.6])

Molecule<>

In [5]:
print('The total number of atoms in system is: ', system.num_atoms)

The total number of atoms in system is:  192


## 优化器的选择

一般在给定初始构象之后，都会先使用优化器对构象进行初步的优化，再进行分子动力学模拟。常用的优化器有Adam和自定义的梯度下降SteepDescent等，如果使用自定义的优化器，还可以设置为局部的优化，比如可以只优化氢原子。

In [6]:
potential = ForceField(system, parameters='TIP3P')

opt = Adam(system.trainable_params(), 1e-3)
mini = Sponge(system, potential, opt)

run_info = RunInfo(50)
mini.run(500, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-08-15 10:03:14
[MindSPONGE] Step: 0, E_pot: 1874.9269
[MindSPONGE] Step: 50, E_pot: 219.15038
[MindSPONGE] Step: 100, E_pot: -1060.1877
[MindSPONGE] Step: 150, E_pot: -1696.2368
[MindSPONGE] Step: 200, E_pot: -2013.583
[MindSPONGE] Step: 250, E_pot: -2225.9272
[MindSPONGE] Step: 300, E_pot: -2384.5051
[MindSPONGE] Step: 350, E_pot: -2534.3154
[MindSPONGE] Step: 400, E_pot: -2589.748
[MindSPONGE] Step: 450, E_pot: -2608.951
[MindSPONGE] Finished simulation at 2023-08-15 10:03:19
[MindSPONGE] Simulation time: 5.14 seconds.
--------------------------------------------------------------------------------


## 控温控压

使用优化器对分子系统进行初步的优化之后，就可以使用UpdaterMD去定义一些分子动力学过程。常用的配置参数有：迭代步长、初始速度、积分器、控温算法、控压算法和约束算法等。前者讲到我们可以使用系统、优化器和迭代器三者来定义一个Sponge对象，此外我们还可以先使用WithEnergyCell来封装一个能量函数，然后用能量和优化器的方法来定义一个Sponge对象。这么做的好处是，我们可以在能量函数中增加bias偏置项，用于实现一些软约束算法，或者是增强采样算法，在如下案例中，我们添加了一个自定义的球形谐振子势软约束。

In [7]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.shape, system.atom_mass)

updater = UpdaterMD(
    system=system,
    time_step=1e-3,
    velocity=velocity,
    integrator='velocity_verlet',
    temperature=300,
    thermostat='langevin',
    constraint='all-bonds'
)

sim = WithEnergyCell(system, potential, bias=SphericalRestrict(radius=1.0, center=[0, 0, 0]))
md = Sponge(sim, optimizer=updater)

run_info = RunInfo(200)
cb_h5md = WriteH5MD(system, 'tutorial_c02.h5md', save_freq=200, write_velocity=True, write_force=True)

md.run(2000, callbacks=[run_info, cb_h5md])

[MindSPONGE] The settle constraint is used for the molecule system.
[MindSPONGE] Started simulation at 2023-08-15 10:07:39
[MindSPONGE] Step: 0, E_pot: 6647.012, E_kin: 676.11835, E_tot: 7323.1304, Temperature: 283.83365
[MindSPONGE] Step: 200, E_pot: -968.53015, E_kin: 2078.7773, E_tot: 1110.2472, Temperature: 1312.4381
[MindSPONGE] Step: 400, E_pot: -1459.1447, E_kin: 1066.3557, E_tot: -392.78894, Temperature: 673.2447
[MindSPONGE] Step: 600, E_pot: -1692.3256, E_kin: 698.86975, E_tot: -993.4558, Temperature: 441.23212
[MindSPONGE] Step: 800, E_pot: -1946.2815, E_kin: 532.20245, E_tot: -1414.0791, Temperature: 336.00653
[MindSPONGE] Step: 1000, E_pot: -2132.5386, E_kin: 429.04474, E_tot: -1703.4939, Temperature: 270.8778
[MindSPONGE] Step: 1200, E_pot: -2229.7974, E_kin: 341.96985, E_tot: -1887.8275, Temperature: 215.90298
[MindSPONGE] Step: 1400, E_pot: -2359.7773, E_kin: 379.95728, E_tot: -1979.8201, Temperature: 239.8864
[MindSPONGE] Step: 1600, E_pot: -2423.409, E_kin: 316.2953, 